# db

### interface sql directly

In [17]:
import psycopg2
from heagVehicleLivedataUtils_sar23b.apiUtils import get_current_vehicle_data_as_dataframe

# Database connection settings
DB_SETTINGS = {
    'dbname': 'mydatabase',
    'user': 'user',
    'password': 'password',
    'host': 'localhost',  
    'port': 5432
}

def get_db_connection():
    conn = psycopg2.connect(**DB_SETTINGS)
    return conn

# test db connection
with get_db_connection() as conn, conn.cursor() as cursor:
    print("all fine")

In [ ]:
with get_db_connection() as conn, conn.cursor() as cursor:
    vehicle_line_list = get_current_vehicle_data_as_dataframe().reindex(columns=['lineId']).reset_index("vehicleId").values.tolist()
    for [vehicle, line] in vehicle_line_list:
        cursor.execute(f"INSERT INTO vehicledata (vehicleId, lineId) VALUES ({vehicle}, {line})")
    conn.commit()

In [9]:
with get_db_connection() as conn, conn.cursor() as cursor:
    cursor.execute("SELECT * FROM vehicleservice ()")
    trams = cursor.fetchall()
    
trams

[]

In [18]:
# add test relation
with get_db_connection() as conn, conn.cursor() as cursor:
    cursor.execute("INSERT INTO vehicleservice (time, vehicleid, lineid, category, direction, status, latitude, longitude, bearing, type) VALUES ('2024-11-30 22:43:57+01:00', 10, 7, 1, 'test', 3, 12.3, 3.14, 50, 1)")

### work with sqlalchemy

In [1]:
from sqlalchemy import create_engine
from heagVehicleLivedataUtils.apiUtils import get_current_vehicle_data_as_dataframe
import pandas as pd

# Database connection settings
DB_SETTINGS = {
    'dbname': 'mydatabase',
    'user': 'user',
    'password': 'password',
    'host': 'localhost',  
    'port': 5432
}

engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{dbname}'.format(**DB_SETTINGS))

# test connection
with engine.connect() as conn:
    print(pd.read_sql("SELECT * FROM vehicleservice", conn))
    #print(pd.read_sql("SELECT * FROM users", conn))
    

Empty DataFrame
Columns: [identifier, time, vehicleid, lineid, category, direction, status, latitude, longitude, bearing, type]
Index: []


In [68]:
# read contents of database
with engine.connect() as conn:
    print(pd.read_sql("SELECT * FROM vehicleservice", conn))

Empty DataFrame
Columns: [identifier, time, vehicleid, lineid, category, direction, status, latitude, longitude, bearing, type]
Index: []


#### test dataframe .to_sql

In [63]:
# try adding test dataframe to users - >
c = ["name", "age"]
d = [("try", 123)]
df = pd.DataFrame(d, columns=c)
#df.to_sql("users",con=engine, if_exists='append',index=False) 
with engine.connect() as conn, conn.begin():
    df.to_sql("users",con=conn, if_exists='append',index=False)

In [69]:
# try adding test to vehicleservice
import datetime
c = ["time", "vehicleid", "lineid", "category", "direction", "status", "latitude", "longitude", "bearing", "type"]
d = [(datetime.datetime(2024,1,21,12,30,1), 20, 89, 1, 'zwei', 4, 12.4, 2.71, 25, 5)]
df = pd.DataFrame(d, columns=c)
df.to_sql("vehicleservice",con=engine, if_exists='append', index=False)

1

In [5]:
get_current_vehicle_data_as_dataframe().to_sql("vehicleservice",con=engine, if_exists='append', index_label=['time','vehicleid'])

72

das problem war die indxe als seperate spalte nochmal mitzugeben

#### other methods

In [26]:
# hier ist ausprobieren wie sql queries direkt funktionieren
with engine.connect() as conn, conn.begin():
    pass #TODO figure out how this works -> see e.g. pandas DF.to_sql examples
    #conn.execute("DROP TABLE IF EXISTS vehicleservice") # ist nicht iwi beim cursor??

ObjectNotExecutableError: Not an executable object: 'DROP TABLE IF EXISTS vehicleService'